In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
import statsmodels.formula.api as smf
# from linearmodels.panel import PanelOLS

In [2]:
wf = pd.read_csv("data/wf.csv")
city_yb = pd.read_csv("data/city_yb.csv")

In [3]:
wf = pd.read_csv("data/wf.csv")
city_yb = pd.read_csv("data/city_yb.csv")

wf["temp2"] = wf["temp"] ** 2
wf["l_aqi"] = np.log(1 + wf["aqi"])
wf["l_pm"] = np.log(1 + wf["pm"])
wf2020 = wf[(wf["daynum"] >= 8401) & (wf["daynum"]<= 8461)].dropna(
    subset = ['aqi', 'pm']
)
wf2020['cities'] = wf2020['city_code'].astype('category')
wf2020['days'] = wf2020['daynum'].astype('category')
wf2020 = pd.get_dummies(wf2020, drop_first=True)

fixed = ['treat']
for col in wf2020.columns:
    if 'cities' in col or 'days' in col:
        fixed.append(col)
        
weather = ['prec', 'snow', 'temp', 'temp2']
out = ["aqi", "l_aqi", "pm", "l_pm"]

## DOUBLE ML

In [4]:
treated = wf2020[wf2020['treat'] == 1]
treated = treated[['daynum', 'city_code']].groupby('city_code')
first = treated.apply(lambda x: x.sort_values(by = 'daynum', ascending=True).head(1))
first = {city:day for day, city in first.values}

In [5]:
wf2020 = wf2020.assign(first = [first.get(city, 0) for city in wf2020['city_code']])
wf2020['first'] = wf2020['first'].astype('category')
wf2020 = pd.get_dummies(wf2020, drop_first=True)

In [6]:
wf2020

,Unnamed: 0,date,prec,snow,temp,aqi,co,no2,o3,pm10,...,first_8433,first_8434,first_8435,first_8436,first_8437,first_8438,first_8439,first_8440,first_8441,first_8456
365,366,20200101,0.433082,51.239401,-6.236673,52.416668,0.671667,44.291668,13.708333,47.125000,...,0,0,0,0,0,0,0,0,1,0
366,367,20200102,0.395048,51.715540,-3.637026,70.583336,1.002500,54.833332,10.500000,68.208336,...,0,0,0,0,0,0,0,0,1,0
367,368,20200103,0.438016,54.323934,-0.863055,70.083336,1.220000,64.250000,15.958333,77.333336,...,0,0,0,0,0,0,0,0,1,0
368,369,20200104,0.912422,61.261527,-2.018923,61.291668,1.222500,62.500000,24.291666,72.125000,...,0,0,0,0,0,0,0,0,1,0
369,370,20200105,46.815258,37.556810,0.916044,86.583336,1.159583,59.916668,12.250000,69.833336,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144413,144414,20200226,96.312215,46.689595,1.266521,130.958328,0.997917,26.875000,31.083334,92.000000,...,0,0,0,0,0,0,0,0,0,0
144414,144415,20200227,4.975817,41.940740,1.981405,64.291664,1.075000,21.166666,67.291664,108.833336,...,0,0,0,0,0,0,0,0,0,0
144415,144416,20200228,37.486767,112.075071,1.805855,33.444443,0.655556,12.000000,69.444443,41.222221,...,0,0,0,0,0,0,0,0,0,0
144416,144417,20200229,105.321617,42.643838,1.459408,27.291666,0.587500,12.583333,60.541668,26.000000,...,0,0,0,0,0,0,0,0,0,0
